# Recommendation Subsystems and Supersystems

#### Go read this

This winter school explores the structure of recommender systems using the hybridisation taxonomy introduced by R. Burke in
- "Hybrid Recommender Systems: Survey and Experiments", 2002, article in "User Modeling and User-Adapted Interaction"
- "Hybrid Web Recommender Systems", 2007, book chapter in "The Adaptive Web"

which is still used in literatures reviews as recently as E. Cano, M. Morisio (Jan 2019) and Wikipedia (Jan 2020).

It also borrows heavily from the chapter on recommendations in Ilya Katsov's "Introduction to Algorithmic Marketing" (2018).


#### Turtles all the way down

Recommender Hybridisation typically embeds recommender systems into larger recommender systems.

A _holon_ is a whole system that is also part of a larger system. A holon is distinct from the other systems that are also part of the larger system. A _holarchy_ is a hierarchy of holons; holarchies clarify the levels of embeddedness at which individual systems operate, the subsystems they are composed of, and the supersystems they are parts of.

Recommender Systems are starting to become complicated enough that they exhibit holarchic structure: They
- are "wholes" made up of multiple systems 
    - existing at multiple levels of embeddedness
    - interacting with one another within these levels
- can exist as "parts" within larger (hybrid) recommender systems

This winter school explores these "whole" and "part" aspects of Recs systems.

In [1]:
# python

# Components of a Recommender System ("Whole" aspect)

Recs Systems can be conceived of as having three sequential "subsystems":

1. candidate generation: This subsystem (typically a ML model) shortlists candidates to rank/recommend
2. ranking / scoring: This subsystem (typically a sorting system) ranks/scores these candidates
3. re-ranking / re-scoring: A final model (typically a rules-based system) re-ranks/re-scores these candidates

In the Holarchic structure, these three sequential subsystems exist at the same level of embeddedness and interact with one another (e.g. the candidates generated by the first subsystem feed into the second subsystem). Each of these systems can contain further subsystems (e.g. candidate generation can be a hybrid of two recommenders).

Burke does not explicitly consider subsystems of a recs system as such, instead considering the time-ordered _phases_ of a recs system,

1. training
2. candidate generation
3. scoring

Burke's diagrams accompanying this phase-based view clearly show how recommenders are embedded into the candidate generation and scoring components of hybrid recommender systems.

## Candidate Generation Subsystem

Typical examples include Collaborative Filtering, Content Filtering, and (less commonly) Knowledge-based Filtering. These candidate generation systems have complementary 'PRO's and 'CON's (which is why hybridising them works in the first place).

### Collaborative Filtering

Given ratings of many products by many users, and given a set of users "similar" to a given user, a "rating from similar users" can be _aggregated_ for each product, and a set of candidates determined from this aggregate rating from similar users. This is a "user-item" Collaborative Filtering algorithm. There are also item-item CF algorithms, which work somewhat similarly.

PROS:
- promotes new behaviors in the user (discovery)
- bla

CONS:
- must have ratings of many products by many users (cold start)
- bla

In [2]:
# python

### Content Filtering

Given features of products, and given a user's ratings of other products, a _binary classifer_ that fits the user's rating behavior can be generated, and used to determine a set of candidates.

PROS:
- bla
- bla

CONS:
- must have a user's ratings history (cold start)
- only promotes user's observed behavior (no discovery)

In [3]:
# python

# scikit-learn a simple classifier

### Knowledge-based Filtering

Given explicit knowledge of a user's needs and preferences, and explicit knowledge of how certain products or product features meet these needs, a set of candidates can be _logically inferred_ (via logic programming or otherwise).

PROS:
- no cold start
- human-interpetable inference rules

CONS:
- new inference rules (knowledge) are usually hand-crafted
- only promotes behavior we already know about (no discovery)

In [4]:
from problog.program import PrologString
from problog import get_evaluatable

In [5]:
model = """
% Explicit knowledge about which products satisfy which needs

% Rule: if user has a given need, then this product should be candidate
trousers_923657(X) :- needs_trousers(X).
trousers_984520(X) :- needs_trousers(X).
dress_83745(X) :- needs_dress(X).

% Manual Rule: People who never shop in female categories don't want dresses

needs_trousers(X) :- shops_female(X).
needs_dress(X) :- shops_female(X).
needs_trousers(X) :- not(shops_female(X)).
% needs_dress(X) :- not(shops_female(X)).  <-- not recommended!


% Manual Rule: get rid of excess stcok of ugly trousers by
% recommending them to people who have poor taste in clothes

ugly_trousers(X) :- needs_trousers(X), not(shops_female(X)).
"""

queries = """
shops_female(1).
not(shops_female(2)).

query(trousers_923657(1)).
query(trousers_984520(1)).
query(ugly_trousers(1)).
query(dress_83745(1)).

query(trousers_923657(2)).
query(trousers_984520(2)).
query(ugly_trousers(2)).
query(dress_83745(2)).
"""

problog_string = model+queries

result = get_evaluatable().create_from(PrologString(problog_string)).evaluate()

In [6]:
result

{trousers_923657(1): 1.0,
 trousers_984520(1): 1.0,
 ugly_trousers(1): 0.0,
 dress_83745(1): 1.0,
 trousers_923657(2): 1.0,
 trousers_984520(2): 1.0,
 ugly_trousers(2): 1.0,
 dress_83745(2): 0.0}

## Ranking / Scoring Subsystem

Recommendation systems typically sort their candidates into an ordered list.

Given a scoring function $f({\rm candidate}) \to {\rm score}$, this system performs two operations
- Computes the scores `scores = map(f, candidate_shortlist)`
- Ranks the candidates by their scores `sort(by=score, zip(scores, candidate_shortlist))`

The choice of a scoring function $f$ need not be informed by the scores used by the candidate generator subsystem. The scoring function $f$ may depend on an individual user's context.

### Scoring

TODO

### Sorting

TODO

## Re-ranking/Re-scoring Subsystem

The point of a re-scoring/re-ranking subsystem is that the scoring/ranking subsystem considers candidates _individually_ while the second pass allows us to fine tune the candidates as a _group_. We might care about visitor-centric properties of the recommendations (e.g. Promotions / Blacklists / Whitelists / Price Anchoring) or about "digital storefront" concerns (e.g. coordination of multiple carousels, manual curation, carousel slot takeovers) or even about ML concerns (e.g. exploration of 'new' candidates for recs, explicit unbiasing, ...)

### Rules

Rules-based subsystems in Recs can be described by a BDD-style description:
    ```
    GIVEN <CONTEXT> [AND <CONTEXT> ...]*,
    WHEN <TRIGGER>,
    THEN <BEHAVIOR>.
    ```
e.g.
    - `GIVEN user.category_preference=socks, WHEN (...) THEN (PROMOTE category=shoes)`
    - `GIVEN seed.price AS minimum, WHEN (...), THEN (BLACKLIST price < minimum)`  -- "don't downsell"
    - `GIVEN seed.category=X, WHEN (...), THEN (CURATE category=Y)`  -- "complete the look"

The explicit distinction between context and trigger means that we can trigger rule-checks at different times, to create a prioritised sequence of rules (which enables us to use rules that "do not commute").

Unlike Re-ranking/Re-scoring systems, candidate generation systems (above) do not need to supply a scoring/ranking of their candidates. The output of a (Knowledge-based Filtering)  candidate generation system  needs to be ranked and sorted by the Ranking / Scoring subsystem; the output of a re-ranking/re-scoring system is already ranked and sorted.

In [7]:
# python

# Hybridising Recommender Systems ("Part aspect")

Burke distinguishes seven ways of hybridising Recs systems:

1. Weighted: The score of different recommendation components are combined numerically.
2. Switching: The system chooses among recommendation components and applies the selected one.
3. Mixed: Recommendations from different recommenders are presented together.
4. Feature Combination: Features derived from different knowledge sources are combined together and given to a single recommendation algorithm.
5. Feature Augmentation: One recommendation technique is used to compute a feature or set of features, which is then part of the input to the next technique.
6. Cascade: Recommenders are given strict priority, with the lower priority ones breaking ties in the scoring of the higher ones.
7. Meta-level: One recommendation technique is applied and produces some sort of model, which is then the input used by the next technique.

We will discuss each hybridisation in more detail below.

Katsov distinguishes _blending_ hybridisations, which perform _ensemble learning_ over recommenders, from the simpler hybridisations (Switching, Mixing). The important insight here is that _hybridisation is ensemble learning_. We will find all of the usual techniques: bootstrap aggregating == ???, gating == switching, ...

As in any taxonomy, the taxons are arbitrary conceptual distinctions. Recommender systems did not evolve in order to manifest a taxonomy, the taxonomy was created to classify recommender system hybrids. By analogy with biology, recall that cultural evolution is the driving force behind the emergence of new species of Recs systems. In the biosphere, hybridisation (an explicit blurring of species distinctions) is feature of polyploidy. Burke's taxonomy _of hybridisation_ would consider genetic hybrids as "Mixing" hybrids (each parent 'recommends' one set of chromosomes, which are presented together in the genotype).

This taxonomy does not account for the existence of subsystems of recommendation systems: Does hybridisation occur
- inside the candidate generation system?
- between candidate generation and scoring?
- inside the scoring system?
- between scoring and re-scoring?
- inside re-scoring system?

## Weights (aka Stacking)

In ensemble learning, this form of hybridisation is known as stacking (which is how we will refer to this hybridisation from now on). Given scores $s_{ri}$ for multiple items (indexed by $i$) from multiple recommenders (indexed by $r$), stacking performs:
$$
s_i = f(s_{0i}, s_{1i}, \ldots, s_{ni})
$$
where $f$ is typically trained by some regression model, e.g. linear regression:
$$
s_i = \sum_r \alpha_r s_{ri}
$$

In order that the scores be sensibly combined, they must be "similar enough", under the combination. This is similarity is not only in score magnitudes (which would be trivially addressed by the weights) but also about score variabilities: if the combination is "more sensitive" to one weight of the combination than another, then they are not being combined democratically -- score perturbations of the "more sensitive" system would "weigh more". For instance, if the scores of two recs systems are combined by a linear regression, then an increment $s \to s + \Delta$ in any weight from the first system _should_ have the same effect (up to the coefficients $\alpha_r$ of the regression) as the same increment on a weight from the second system.

This similarity requirement suggests that stacking hybridisations occur as part of, or downstream of, the scoring subsystem of the hybrid recommender. The same scoring function is assigned to the candidates of each recommender, and these scores-per-candidate can be stacked to obtain the output score of the hybrid recommender.

In [8]:
# python

## Switching

Suppose you had three doctors: an oncologist, an ophthalmologist, and a parasitologist. A Switching Recommender would take the ophthalmologist's advice on eye-related issues, the oncologist's advice on cancer-related issues, and the parasitologist's advice on parasite-related issues.

You might want to use collaborative filtering for users where you have the data, and switch to a knowledge-based system for users where you don't. Or use different recs scores for different target audiences, e.g. recommend products driving conversions for new visitors and driving engagement/loyalty metrics for returning visitors.

You might also know that some recommenders perform better than others in different parts of the product catalogue. For example, given that the current PDP is in product category "makeup >> foundation" you might want to switch from the generic recommender to a foundation-specific recommender to upsell or improve the visitor's odds of finding the right color. This specific recommender is smaller (and so easier to train) and might benefit from features (e.g. user skin tone) that the generic recommender may not know how to use.

If the switching logic is not hard coded but machine-learned with a neural network, then this neural network is known as a _gating network_. Note that gating networks serve a distinct purpose than stacking networks: switching between individual "experts" is not the same as blending them.

In [9]:
# python

## Mixing

Mixing shows recommendations from multiple recommenders in a combined display. For instance, Youtube's front page has multiple recommendation carousels. The ranked lists of videos from separate recommenders are shown in separate carousels, but no video is ever shown twice in the combined display.

In [10]:
# python

recs = [["A", "B", "C"], ["D", "E", "B", "A"], ["F", "A", "G", "H"]]

carousel_lenghts = [3, 2, 3]  # lengths of the carousels

## Feature Combination

In [11]:
# python

## Feature Augmentation

Augmenting the features (by value imputation), or augmenting the feature space with new features.

In [12]:
# python

## Cascade

In [13]:
# python

## Meta-Level

In [14]:
# python